In [ ]:
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers.legacy import SGD as SGD_legacy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import argmax
import tensorflow as tf
from sklearn.metrics import accuracy_score
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Definir las rutas para los datos de entrenamiento y validación
train_data_dir = '/kaggle/input/100-bird-species/train'
validation_data_dir = '/kaggle/input/100-bird-species/valid'

In [ ]:
# Tamaño de la imagen y tamaño del lote
img_width, img_height = 28, 28
batch_size = 64

In [ ]:
# Crear un ImageDataGenerator para los datos de entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,         # normalizar valores de píxeles a [0,1]
    shear_range=0.2,         # transformaciones de cizallamiento
    zoom_range=0.2,          # transformaciones de zoom
    horizontal_flip=True     # volteos horizontales
)

In [ ]:
# Crear un ImageDataGenerator para los datos de validación (solo reescalado)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Cargar y preparar los datos de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Cargar y preparar los datos de validación
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Obtener etiquetas verdaderas del generador de validación
true_labels = validation_generator.classes

In [ ]:
num_classes = len(os.listdir(train_data_dir))

In [ ]:
def define_cnn_model(input_shape, num_classes):
    """
    Definir un modelo de Red Neuronal Convolucional (CNN).

    Parámetros:
    - input_shape (tuple): La forma de los datos de entrada (por ejemplo, (img_width, img_height, num_channels)).
    - num_classes (int): El número de clases en la tarea de clasificación.

    Devuelve:
    - model (Sequential): El modelo CNN definido.
    """
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(500, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(num_classes, activation='softmax'))
    
    opt = Adam(learning_rate=0.001)
    #opt = SGD_legacy(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Entrenar el modelo CNN utilizando ImageDataGenerator
model = define_cnn_model((img_width, img_height, 3), num_classes)   # Suponiendo imágenes RGB
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
num_train_samples = len(train_generator.filenames)
num_validation_samples = len(validation_generator.filenames)

In [ ]:
# Establecer la ejecución ansiosa
tf.config.run_functions_eagerly(True)

In [ ]:
# Ajustar el modelo
history = model.fit(
    train_generator,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=num_validation_samples // batch_size,
    #callbacks=[early_stopping]
)

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# Predecir las etiquetas para el conjunto de prueba
predictions = model.predict_generator(test_generator, steps=num_validation_samples // batch_size + 1)
predicted_labels = [argmax(pred) for pred in predictions]

In [ ]:
# Asegurar que las longitudes sean iguales
true_labels = true_labels[:len(predicted_labels)]

In [ ]:
# Evaluar el modelo en el conjunto de prueba
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [ ]:
from keras.models import load_model
model.save('my_model.keras')

In [ ]:
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

# Load the trained model
model = load_model('my_model.keras')

# Load and prepare an image
def prepare_image(file_path, target_size):
    img = load_img(file_path, target_size=target_size)  # Load the image
    img = img_to_array(img)  # Convert the image to a numpy array
    img = np.expand_dims(img, axis=0)  # Add a batch dimension
    img = img / 255.0  # Scale pixel values to [0, 1]
    return img

image_path = '/kaggle/input/100-bird-species/test/AMERICAN COOT/3.jpg'
prepared_img = prepare_image(image_path, target_size=(28, 28))  # Ensure target_size is same as model's input size

# Predict the class
prediction = model.predict(prepared_img)
predicted_class = np.argmax(prediction, axis=1)

# Output the predicted class
print(f"Predicted class: {predicted_class[0]}")